## 1. Imports


In [2]:
import sys
sys.path.append('../')
from visualizations import impact_bar_plots
from evaluation import *
from testing import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
data_path = '../data/results/test_1/'
folders= ['dt','gnb','lgr','gbt']
folders = ['dt']

## 2. DI_plots and CSV

In [ ]:
impact_bar_plots(data_path = '../data/results/test_3/',b_or_w = 'black',folders= ['dt','gnb','lgr','gbt'])
impact_bar_plots(data_path = '../data/results/test_3/',b_or_w = 'white',folders= ['dt','gnb','lgr','gbt'])

In [4]:
impact_csvs(data_path= '../data/results/test_3/',b_or_w = 'black', folders= ['dt','gnb','lgr','gbt'])
impact_csvs(data_path= '../data/results/test_3/',b_or_w = 'white', folders= ['dt','gnb','lgr','gbt'])

Group:  black 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated   8.03    0.00    8.33    8.05   8.03    0.00    8.33    8.05
DP          -34.85  -23.87  -36.30  -36.40 -52.12    0.00  -52.12  -52.12
EO           -0.32   -9.72   -4.31   -0.51   8.03  -10.79    8.33    8.05
EOO         -11.25  -15.22  -12.70   -8.63 -22.91  -18.70  -26.82  -22.08
FPER         -0.82  -14.36   -3.55   -0.28   8.03    0.00    8.33    8.05
ERP           8.03    8.33    8.14    8.35   8.03    8.98    8.33    8.05
A
B
Group:  white 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated  38.49   36.19   37.93   38.49  38.49   36.19   37.93   38.49
DP           38.78   19.93   38.75   38.95  39.11   36.19   39.07   39.07
EO           37.62   35.38   36.73   37.54  38.49 

C:\Users\HaRi\Documents\GitHub\delayedimpact-daadww\notebooks\..\testing.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\Documents\GitHub\delayedimpact-daadww\notebooks\..\testing.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


## 3. FP/TP/TN/FN Ratios

In [5]:
types_ratios_csv(data_path = '../data/results/test_3/',folders= ['dt','lgr','gbt','gnb'])

Classifier:  dt 
 DataFrame: 
 ID        egdpB     egdpW     egeoB     egeoW    egerpB    egerpW   egfprpB  \
Value                                                                         
FN     0.012031  0.054378  0.039452  0.089874  0.096530  0.072126  0.039452   
FP     0.419138  0.064633  0.145775  0.054605  0.061556  0.073072  0.149133   
TN     0.245383  0.175358  0.518747  0.185386  0.602966  0.166919  0.515389   
TP     0.323447  0.705631  0.296027  0.670135  0.238948  0.687883  0.296027   

ID      egfprpW   egtprpB   egtprpW     gsdpB     gsdpW     gseoB     gseoW  \
Value                                                                         
FN     0.057708  0.027420  0.058087       NaN  0.084273  0.096530  0.046999   
FP     0.061833  0.224678  0.061606  0.664522  0.047491  0.061556  0.070234   
TN     0.178158  0.439843  0.178385       NaN  0.192500  0.602966  0.169757   
TP     0.702301  0.308058  0.701922  0.335478  0.675736  0.238948  0.713010   

ID       gserpB    

## 4. Score Distributions

Loading All Scores and splitting it into multiple dataframes

In [ ]:
classifier_dfs = {}
dfs_b = {}
dfs_w = {}
dfs_eg = {}
dfs_gs = {}
for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    df_eg = pd.concat([df.iloc[:,:4],df.filter(like='eg')],axis=1)
    df_gs = pd.concat([df.iloc[:,:4],df.filter(like='gs')],axis=1)
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white
    dfs_eg[f] = df_eg
    dfs_gs[f] = df_gs

Checking if normal distributions:

In [ ]:
for c,df in classifier_dfs.items():
    for col in df:
        print(col)
        data=df[col].dropna(axis=0)
        print(stats.kstest(data, "norm"))

In [ ]:


for c,df in classifier_dfs.items():
    fig,axes = plt.subplots(nrows= 5)
    
    for i in range(5):
        axes[i] = plt.plot(df.iloc[:,i].value_counts().sort_index())
        plt.tight_layout()
        plt.show()
        